# All Test-Train-S50F-DNN  with PipeLine


Baseline Neural Network Model Performance

https://gist.github.com/MaxHalford/9bfaa8daf8b4bc17a7fb7ba58c880675

In [99]:
xseed=8

from numpy.random import seed
seed(xseed)
from tensorflow import set_random_seed
set_random_seed(xseed)

In [100]:
 # Binary Classification with Sonar Dataset: Baseline
import numpy
import pandas

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import models

from keras.utils import np_utils

from keras.layers import Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.constraints import maxnorm


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

import joblib

from keras.utils import to_categorical

import os
from datetime import datetime

In [101]:
def CreateData(dfx,labelCol):
 
 X_df_temp =dfx.drop(columns=['open','high','low','close',labelCol])
 Y_df_temp = dfx[labelCol]

 Xtemp=X_df_temp.to_numpy()
 ytemp=Y_df_temp.to_numpy()

 return X_df_temp,Y_df_temp, Xtemp,ytemp

In [102]:
result_path=os.path.abspath(r'D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\results')
ml_path=os.path.abspath(r'D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\models')

In [103]:
st='2007'
ed='2018'

test_st='2019'
test_ed='2020'


strategyName='long'

In [104]:
filename='ML-Long_S50M15_07ToNow-Train.csv'
filepath=os.path.abspath(f'D:\DataSets\/{filename}')
print(filepath)


D:\DataSets\ML-Long_S50M15_07ToNow-Train.csv


# Train Data 

In [105]:
# load dataset

dataframe = pandas.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
                                                 
print(dataframe.shape)

                     
dataframe.info()

(70693, 22)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70693 entries, 2007-01-03 09:45:00 to 2020-06-10 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    70693 non-null  float64
 1   high                    70693 non-null  float64
 2   low                     70693 non-null  float64
 3   close                   70693 non-null  float64
 4   indy_ma-550             70693 non-null  float64
 5   indy_ma-1100            70693 non-null  float64
 6   indy_hh-550             70693 non-null  float64
 7   indy_ll-550             70693 non-null  float64
 8   indy_mid-550            70693 non-null  float64
 9   indy_hh2-1100           70693 non-null  float64
 10  indy_ll2-1100           70693 non-null  float64
 11  indy_mid2-1100          70693 non-null  float64
 12  indy_macd110-440        70693 non-null  float64
 13  indy_signal110-440-110  70693 non-null  floa

In [106]:
df=dataframe[st:ed]

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62855 entries, 2007-01-03 09:45:00 to 2018-12-28 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    62855 non-null  float64
 1   high                    62855 non-null  float64
 2   low                     62855 non-null  float64
 3   close                   62855 non-null  float64
 4   indy_ma-550             62855 non-null  float64
 5   indy_ma-1100            62855 non-null  float64
 6   indy_hh-550             62855 non-null  float64
 7   indy_ll-550             62855 non-null  float64
 8   indy_mid-550            62855 non-null  float64
 9   indy_hh2-1100           62855 non-null  float64
 10  indy_ll2-1100           62855 non-null  float64
 11  indy_mid2-1100          62855 non-null  float64
 12  indy_macd110-440        62855 non-null  float64
 13  indy_signal110-440-110  62855 non-null  float64
 14  ind

In [107]:
labelName=dataframe.columns[-1]
labelName

'LongSignal'

In [108]:
X_df,y_df,X_train,y_train=CreateData(df,labelName)


In [109]:
X_df.tail(3)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2018-12-28 16:15:00,1076.66,1085.30,1123.2,1022.6,1072.9,1125.0,1022.6,1073.8,-29.61,-24.48,-5.13,43.43,1051.53,1,0,3,2
2018-12-28 16:30:00,1076.60,1085.23,1123.2,1022.6,1072.9,1125.0,1022.6,1073.8,-29.75,-24.56,-5.19,43.98,1050.89,1,0,3,2
2018-12-28 16:45:00,1076.53,1085.16,1123.2,1022.6,1072.9,1125.0,1022.6,1073.8,-29.81,-24.64,-5.17,44.65,1049.59,1,0,3,2


In [110]:
y_df.tail(3)

datetime
2018-12-28 16:15:00    0
2018-12-28 16:30:00    0
2018-12-28 16:45:00    0
Name: LongSignal, dtype: int64

# Test Data

In [111]:
#filename_test=filename
'''
filename_test='ML-Long_S50M15_19ToNow-Train_Demo.csv'
filepath_test=os.path.abspath(f'D:\Dev-Trong\DataSet\{filename_test}')
print(filepath)
dataframe_test = pandas.read_csv(filepath_test,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
df_test=dataframe_test[test_st:test_ed]
'''

df_test=dataframe[test_st:test_ed]

df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7838 entries, 2019-01-02 09:45:00 to 2020-06-10 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    7838 non-null   float64
 1   high                    7838 non-null   float64
 2   low                     7838 non-null   float64
 3   close                   7838 non-null   float64
 4   indy_ma-550             7838 non-null   float64
 5   indy_ma-1100            7838 non-null   float64
 6   indy_hh-550             7838 non-null   float64
 7   indy_ll-550             7838 non-null   float64
 8   indy_mid-550            7838 non-null   float64
 9   indy_hh2-1100           7838 non-null   float64
 10  indy_ll2-1100           7838 non-null   float64
 11  indy_mid2-1100          7838 non-null   float64
 12  indy_macd110-440        7838 non-null   float64
 13  indy_signal110-440-110  7838 non-null   float64
 14  indy

In [112]:
dfOHLC=df_test.loc[:,['open','high','low','close']]

In [113]:
Xtest_df,ytest_df,X_test,y_test=CreateData(df_test,labelName)

In [114]:
Xtest_df.tail(5)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2020-06-10 15:45:00,887.42,847.35,977.9,834.2,906.05,977.9,715.3,846.6,56.29,40.88,15.40,45.45,952.83,3,1,4,4
2020-06-10 16:00:00,887.55,847.54,977.9,834.2,906.05,977.9,715.3,846.6,56.36,41.18,15.18,45.81,952.77,3,1,4,4
2020-06-10 16:15:00,887.68,847.74,977.9,834.2,906.05,977.9,715.3,846.6,56.43,41.48,14.95,46.25,952.82,3,1,4,4
2020-06-10 16:30:00,887.85,847.94,977.9,834.2,906.05,977.9,715.3,846.6,56.54,41.78,14.76,46.64,952.65,3,1,4,4
2020-06-10 16:45:00,888.02,848.15,977.9,834.2,906.05,977.9,715.3,846.6,56.64,42.07,14.57,47.13,952.71,3,1,4,4


In [115]:

ytest_df.tail(5)

datetime
2020-06-10 15:45:00    1
2020-06-10 16:00:00    1
2020-06-10 16:15:00    1
2020-06-10 16:30:00    1
2020-06-10 16:45:00    1
Name: LongSignal, dtype: int64

In [116]:
#for categorical_crossentropy and softmax
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(62855, 17)
(62855, 2)
(7838, 17)
(7838, 2)


# Model Creation

In [119]:
h1=17
h2=17
h3=0

input_n=X_train.shape[1]

xpoch=105

#Epoch 00107: early stopping
xbatch=32

kernelInit='he_uniform'
#kernelInit='he_normal'
#kernel_initializer="glorot_uniform"  #default

patience_from_epoch=round( xpoch*0.5)

print("Input-N =",input_n)
print('Epochs=',xpoch)
print('Batch_size=',xbatch)
print('Patience from epochs',patience_from_epoch )

modelSelect=''

modelHD_Title=f'{h1}x{h2}x{h3}_epo{xpoch}_btch{xbatch}'
print(modelHD_Title)

Input-N = 17
Epochs= 105
Batch_size= 32
Patience from epochs 52
17x17x0_epo105_btch32


In [120]:
def ModelByAllTuning():
    model_x=Sequential()
    
    param_dropout=0.02
    vaMaxNorm_WC=4
    
    l2_val=0.00100000
    
     #input
    model_x.add(Dropout(param_dropout, input_shape=(input_n,)))
    
    #layer#1
    model_x.add(Dense(h1,activation="relu", kernel_constraint=maxnorm(vaMaxNorm_WC),kernel_regularizer=l2(l2_val)))
    #model_x.add(Dense(h1, kernel_initializer=kernelInit,activation="relu", kernel_constraint=maxnorm(vaMaxNorm_WC),kernel_regularizer=l2(l2_val)))
    model_x.add(Dropout(param_dropout))
    
    #layer#2
    model_x.add(Dense(h2, activation="relu", kernel_constraint=maxnorm(vaMaxNorm_WC),kernel_regularizer=l2(l2_val)))
    #model_x.add(Dense(h2,kernel_initializer=kernelInit, activation="relu", kernel_constraint=maxnorm(vaMaxNorm_WC),kernel_regularizer=l2(l2_val)))
    model_x.add(Dropout(param_dropout))
    
    if y_train.ndim==2: # Convert to multil classification
     print("# Convert to multil classification")
     model_x.add(Dense(2, activation='softmax'))
     model_x.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    else :
     model_x.add(Dense(1, activation="sigmoid"))  
     model_x.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    

    
    return model_x

In [121]:

#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience_from_epoch)

estimator_steps=[]

#estimator_steps.append(('standardize', StandardScaler()))
estimator_steps.append(('minmax_range', MinMaxScaler()))

modelHD_Title=f'{modelHD_Title}_modeAllTunine'
estimator_steps.append(('mlp', KerasClassifier(build_fn=ModelByAllTuning, epochs=xpoch, batch_size=xbatch)))

#estimator_steps.append(('mlp', KerasClassifier(build_fn=ModelByAllTuning, epochs=xpoch, batch_size=xbatch,callbacks=[es])))


mypipeline=Pipeline(estimator_steps)

In [122]:
mypipeline

Pipeline(memory=None,
         steps=[('minmax_range', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000234ED9AE348>)],
         verbose=False)

In [123]:
mypipeline.fit(X_train,y_train)

# Convert to multil classification

Epoch 1/105
62855/62855 [==============================] - 2s 24us/step - loss: 0.4244 - accuracy: 0.8194
Epoch 2/105
62855/62855 [==============================] - 1s 22us/step - loss: 0.3830 - accuracy: 0.8363
Epoch 3/105
62855/62855 [==============================] - 1s 22us/step - loss: 0.3711 - accuracy: 0.8420
Epoch 4/105
62855/62855 [==============================] - 1s 23us/step - loss: 0.3656 - accuracy: 0.8433
Epoch 5/105
62855/62855 [==============================] - 2s 31us/step - loss: 0.3556 - accuracy: 0.8505
Epoch 6/105
62855/62855 [==============================] - 2s 34us/step - loss: 0.3499 - accuracy: 0.8529
Epoch 7/105
62855/62855 [==============================] - 2s 35us/step - loss: 0.3439 - accuracy: 0.8571
Epoch 8/105
62855/62855 [==============================] - 3s 40us/step - loss: 0.3376 - accuracy: 0.8612
Epoch 9/105
62855/62855 [==============================] - 2s 36us/step - loss: 0.3359 - accuracy: 0.8624
Epoch 10/1

62855/62855 [==============================] - 2s 38us/step - loss: 0.2724 - accuracy: 0.8939
Epoch 76/105
62855/62855 [==============================] - 2s 38us/step - loss: 0.2741 - accuracy: 0.8936
Epoch 77/105
62855/62855 [==============================] - 2s 36us/step - loss: 0.2709 - accuracy: 0.8962
Epoch 78/105
62855/62855 [==============================] - 2s 39us/step - loss: 0.2718 - accuracy: 0.8946 0s - loss: 0.2720 - ac
Epoch 79/105
62855/62855 [==============================] - 2s 37us/step - loss: 0.2699 - accuracy: 0.8949
Epoch 80/105
62855/62855 [==============================] - 2s 32us/step - loss: 0.2729 - accuracy: 0.8939
Epoch 81/105
62855/62855 [==============================] - 2s 34us/step - loss: 0.2698 - accuracy: 0.8967
Epoch 82/105
62855/62855 [==============================] - 2s 31us/step - loss: 0.2719 - accuracy: 0.8951
Epoch 83/105
62855/62855 [==============================] - 2s 31us/step - loss: 0.2700 - accuracy: 0.8962
Epoch 84/105
62855/62855 [=

Pipeline(memory=None,
         steps=[('minmax_range', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000234ED9AE348>)],
         verbose=False)

In [124]:
from sklearn.metrics import accuracy_score

In [125]:
mypipeline

Pipeline(memory=None,
         steps=[('minmax_range', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000234ED9AE348>)],
         verbose=False)

In [127]:
yPredicted =mypipeline.predict(X_test)
predictions = [value for value in yPredicted]


In [128]:
if y_train.ndim==2:
 y_test = [numpy.argmax(y, axis=None, out=None) for y in y_test]
 print(y_test[:10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [129]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.80%


In [130]:
df_predictResult=pandas.DataFrame({'y_Actual':y_test,'y_Predicted':yPredicted.ravel()},index=df_test.index)
df_predictResult.tail(5)

,y_Actual,y_Predicted
datetime,,
2020-06-10 15:45:00,1,1
2020-06-10 16:00:00,1,1
2020-06-10 16:15:00,1,1
2020-06-10 16:30:00,1,1
2020-06-10 16:45:00,1,1


In [131]:
yPredicted_prob = mypipeline.predict_proba(X_test)
print(yPredicted_prob.shape)
yPredicted_prob[:5]


(7838, 2)


array([[9.9980897e-01, 1.9109686e-04],
       [9.9980932e-01, 1.9070132e-04],
       [9.9980956e-01, 1.9048979e-04],
       [9.9980980e-01, 1.9021153e-04],
       [9.9981040e-01, 1.8964491e-04]], dtype=float32)

In [132]:
df_predictProb=pandas.DataFrame(data=yPredicted_prob,columns=['Prob0','Prob_1'],index=df_test.index)
print(df_predictProb.shape)
df_predictProb.round(3).tail()


(7838, 2)


,Prob0,Prob_1
datetime,,
2020-06-10 15:45:00,0.223,0.777
2020-06-10 16:00:00,0.228,0.772
2020-06-10 16:15:00,0.234,0.766
2020-06-10 16:30:00,0.239,0.761
2020-06-10 16:45:00,0.244,0.756


In [133]:
df_predictAll=pandas.concat([dfOHLC,df_predictResult,df_predictProb],axis=1)
df_predictAll.head()

,open,high,low,close,y_Actual,y_Predicted,Prob0,Prob_1
datetime,,,,,,,,
2019-01-02 09:45:00,1039.0,1046.4,1038.2,1044.3,0,0,0.999809,0.000191
2019-01-02 10:00:00,1044.4,1045.0,1039.7,1041.7,0,0,0.999809,0.000191
2019-01-02 10:15:00,1041.7,1044.1,1041.6,1043.0,0,0,0.999810,0.000190
2019-01-02 10:30:00,1042.9,1047.0,1042.6,1045.0,0,0,0.999810,0.000190
2019-01-02 10:45:00,1045.3,1045.8,1043.7,1044.5,0,0,0.999810,0.000190


In [134]:
from sklearn import metrics
pandas.DataFrame(metrics.confusion_matrix(y_test, predictions),columns=['Acutal_SideWay',f'Actual_{strategyName}'],index=['Pred_SideWay',f'Pred_{strategyName}'])

,Acutal_SideWay,Actual_long
Pred_SideWay,4446,203
Pred_long,1302,1887


In [135]:
from sklearn.metrics import classification_report
print(classification_report(y_test,yPredicted))

              precision    recall  f1-score   support

           0       0.77      0.96      0.86      4649
           1       0.90      0.59      0.71      3189

    accuracy                           0.81      7838
   macro avg       0.84      0.77      0.79      7838
weighted avg       0.83      0.81      0.80      7838



In [136]:


buildtime=datetime.now().strftime('%d%m%y_%H%M')

result_filename=f'{result_path}\\{strategyName}_MLP-Pipeline_{modelHD_Title}_tn{st}-{ed}_ts{test_st}-{test_ed}_{buildtime}_resultA1.csv'

print(result_filename)



D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\results\long_MLP-Pipeline_17x17x0_epo105_btch32_modeAllTunine_tn2007-2018_ts2019-2020_030720_1631_resultA1.csv


In [137]:
df_predictAll.to_csv(result_filename)

# Save Model

In [138]:


model_filename=f'{ml_path}\\A1_{strategyName}_MLP-Pipeline_{modelHD_Title}_tn{st}-{ed}_ts{test_st}_{buildtime}'

print(model_filename)

D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\models\A1_long_MLP-Pipeline_17x17x0_epo105_btch32_modeAllTunine_tn2007-2018_ts2019_030720_1631


In [139]:

model_step = mypipeline.steps.pop(-1)[1]

print(model_step)

print(model_step.model)

In [140]:
print(mypipeline)

Pipeline(memory=None,
         steps=[('minmax_range',
                 MinMaxScaler(copy=True, feature_range=(0, 1)))],
         verbose=False)


In [141]:

joblib.dump(mypipeline, f'{model_filename}_pipeline.pkl')


['D:\\JupyterCode\\ML-S50F\\Lab-S5-F-DNN\\models\\A1_long_MLP-Pipeline_17x17x0_epo105_btch32_modeAllTunine_tn2007-2018_ts2019_030720_1631_pipeline.pkl']

In [142]:
models.save_model(model_step.model,f'{model_filename}_model.h5')